# 使用torch.nn来创建神经网络
上一讲已经讲过了autograd，nn包依赖autograd包来定义模型并求导。 一个nn.Module包含各个层和一个forward(input)方法，该方法返回output。

神经网络的典型训练过程如下：

1. 定义包含一些可学习的参数(或者叫权重)神经网络模型；
2. 在数据集上迭代；
3. 通过神经网络处理输入；
4. 计算损失(输出结果和正确值的差值大小)；
5. 将梯度反向传播回网络的参数；
6. 更新网络的参数，主要使用如下简单的更新原则： weight = weight - learning_rate * gradient

In [10]:
# 定义网络
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels,
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [11]:
# 查看可被学习的参数
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [12]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0180, -0.0298, -0.0364,  0.0800, -0.0202, -0.1175,  0.0293, -0.0931,
          0.1471, -0.0182]], grad_fn=<AddmmBackward0>)


In [13]:
# 将所有参数梯度清零
net.zero_grad()
print(torch.randn(1, 10))

out.backward(torch.randn(1, 10))


tensor([[ 0.2686, -0.2749,  0.2622, -0.5239,  0.0759,  0.4575,  0.2495, -0.8110,
          0.2855,  0.5966]])


# 损失函数

In [14]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)


tensor(0.9534, grad_fn=<MseLossBackward0>)


In [16]:
# 梯度反向传播
print(loss.grad_fn)

net.zero_grad()     # 清除梯度

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0002,  0.0146, -0.0004,  0.0008, -0.0073,  0.0081])


In [17]:
# 手动更新权重
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)


In [18]:
# 使用优化器
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.01)
# 在循环中
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()

# 定义一个网络
PyTorch中已经为我们准备好了现成的网络模型，只要继承nn.Module，并实现它的forward方法，PyTorch会根据autograd，自动实现backward函数，在forward函数中可使用任何tensor支持的函数，还可以使用if、for循环、print、log等Python语法，写法和标准的Python写法一致


In [4]:
import torch.nn as nn
import torch
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        # 先执行父类构造
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.fc1 = nn.Linear(1350,10)

    def forward(self, x):
        print(x.size())
        x = F.relu(self.conv1(x))
        print(x.size())
        x = F.max_pool2d(x, (2,2))
        x = F.relu(x)
        print(x.size())
        x = x.view(x.size()[0], -1)
        print(x.size())
        x = self.fc1(x)
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1350, out_features=10, bias=True)
)


In [5]:
input = torch.randn(1, 1, 32, 32) # 这里的对应前面fforward的输入是32
out = net(input)
out.size()

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])


torch.Size([1, 10])

In [ ]:
net.zero_grad()
out.backward(torch.ones(1,10)) # 反向传播的实现是PyTorch自动实现的，我们只要调用这个函数即可


# 为什么激活函数都是非线性的？
在神经网络的计算过程中，每层都相当于矩阵相乘，无论神经网络有多少层输出都是输入的线性组合，就算我们有几千层的计算，无非还是个矩阵相乘，和一层矩阵相乘所获得的信息差距不大，所以需要激活函数来引入非线性因素，使得神经网络可以任意逼近任何非线性函数，这样神经网络就可以应用到众多的非线性模型中，增加了神经网络模型泛化的特性。


In [1]:
# 初始化一些信息
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
x = torch.linspace(-10,10,60)
x

tensor([-10.0000,  -9.6610,  -9.3220,  -8.9831,  -8.6441,  -8.3051,  -7.9661,
         -7.6271,  -7.2881,  -6.9492,  -6.6102,  -6.2712,  -5.9322,  -5.5932,
         -5.2542,  -4.9153,  -4.5763,  -4.2373,  -3.8983,  -3.5593,  -3.2203,
         -2.8814,  -2.5424,  -2.2034,  -1.8644,  -1.5254,  -1.1864,  -0.8475,
         -0.5085,  -0.1695,   0.1695,   0.5085,   0.8475,   1.1864,   1.5254,
          1.8644,   2.2034,   2.5424,   2.8814,   3.2203,   3.5593,   3.8983,
          4.2373,   4.5763,   4.9153,   5.2542,   5.5932,   5.9322,   6.2712,
          6.6102,   6.9492,   7.2881,   7.6271,   7.9661,   8.3051,   8.6441,
          8.9831,   9.3220,   9.6610,  10.0000])

sigmoid 函数¶
$a=\frac{1}{1+e^{-z}}$ 导数 ：$a^\prime =a(1 - a)$

In [2]:
ax = plt.plot.gca()
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
ax.xaxis.set_ticks_position('bottom')
ax.spines['bottom'].set_position(('data', 0))
ax.yaxis.set_ticks_position('left')
ax.spines['left'].set_position(('data', 0))
plt.plot.ylim((0, 1))
sigmod=torch.sigmoid(x)
plt.plot(x.numpy(),sigmod.numpy())

AttributeError: 'function' object has no attribute 'gca'